<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_LiveloEsfera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Instalação de pacotes necessários
!pip install selenium
!apt-get update # para o ChromeDriver
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,802 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,072 kB]
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,753 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main a

In [2]:
# 2. Importações
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

In [3]:
# 3. Configurações do Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# 4. Função para extrair dados de uma página
def extrair_dados_amazon_livelo():
    driver.get("https://www.livelo.com.br/juntar-pontos/parceiros/amazon/")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    texto = soup.get_text()
    return "Resumo Livelo:\n" + texto[:1000]  # limita para exemplo

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def extrair_esfera():
    options = Options()
    options.add_argument("--headless")  # Remova esta linha para debugar com navegador visível
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)

    try:
        url = "https://www.esfera.com.vc/p/amazon/e000100298"
        driver.get(url)

        # Espera o carregamento do título como indicativo de que o conteúdo principal foi carregado
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-title")))

        # Agora usa BeautifulSoup para extrair os campos HTML indicados
        soup = BeautifulSoup(driver.page_source, "html.parser")

        titulo = soup.find("h1", class_="product-title").text.strip()

        beneficio_section = soup.find("p", string="Beneficio")
        beneficio_div = beneficio_section.find_next("div")
        beneficio_raw = beneficio_div.contents[0].strip()
        beneficio_info = beneficio_div.find("p").text.strip()

        validade_section = soup.find("p", string="Validade")
        validade_text = validade_section.find_next("div").text.strip()

        credito_section = soup.find("p", string="Creditado em")
        credito_text = credito_section.find_next("div").text.strip()

        resultado = f"""
        📦 Benefício {titulo} = {beneficio_raw} ({beneficio_info})
        🗓️ Validade da promoção: Até {validade_text}
        💳 Pontos creditados em até {credito_text}
        Link de acesso: {url}
        """
    except Exception as e:
        resultado = f"❌ Erro ao extrair dados da Esfera: {str(e)}"
    finally:
        driver.quit()

    return resultado.strip()

In [8]:
esfera = extrair_esfera()
esfera

'📦 Benefício Amazon = 1x1 (Ganhe 1 ponto a cada real)\n        🗓️ Validade da promoção: Até 31/12/25\n        💳 Pontos creditados em até 90 dias do recebimento do produto\n        Link de acesso: https://www.esfera.com.vc/p/amazon/e000100298'

Enviar E-mail

In [ ]:
#importações
from email.message import EmailMessage
import os
import smtplib

#definindo dados de envio do e-mail
remetente = os.environ['EMAIL_REMETENTE']
destinatario = os.environ['EMAIL_DESTINATARIO']
assunto = 'Seu resumo diário pontos Amazon'
mensagem = f"""Olá, aqui está o seu resumo dos pontos Amazon:

{esfera}"""

senha = os.environ['SENHA_GMAIL']

#crinado o e-mail
msg = EmailMessage()
msg['Subject'] = assunto
msg['From'] = remetente
msg['To'] = destinatario
msg.set_content(mensagem)

#enviando o e-mail
with smtplib.SMTP_SSL("smtp.gmail.com",465) as email:
  email.login(remetente,senha)
  email.send_message(msg)

print("O email foi enviado com sucesso!")

KeyError: 'EMAIL_REMETENTE'